# Prepear

In [41]:
# https://bamblebam.medium.com/audio-classification-and-regression-using-pytorch-48db77b3a5ec
# The above code will import all the necessary libraries and initialize your device to either CPU or GPU. 

import torchaudio as ta
from torch.utils.data import DataLoader
import numpy as numpy

import numpy as np
import pandas as pd

import tqdm as tqdm

import torch
from torch import nn
import torchaudio

import cnn

if torch.cuda.is_available():
    print('yes')
    device='cuda'
else:
    device='cpu'

import torch
from cnn import CNNetwork
import torchaudio
import os


import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from glob import glob
from sklearn.preprocessing import LabelEncoder
import os
import matplotlib.pyplot as plt
import pylab
import itertools

import nussl
from importlib import reload 
import nussl_fram
reload (nussl_fram)

from nussl_fram import dataset_v3
from sklearn.model_selection import train_test_split

Device is cpu


# Train

In [55]:
nussl.utils.seed(0)  # make sure this does the same thing each time

stft_params = nussl.STFTParams(window_length=256, hop_length=64)
tfm = nussl.datasets.transforms.Compose([
    nussl.datasets.transforms.PhaseSensitiveSpectrumApproximation(),
    nussl.datasets.transforms.MagnitudeWeights(),
    nussl.datasets.transforms.ToSeparationModel()
])

path = os.path.join(os.getcwd(), "audio")
# TODO - update to 44100
uds = dataset_v3(path, sample_rate=8000, stft_params=stft_params, transform=tfm)
print(f"There are {len(uds)} samples in the data")

There are 40 samples in the data


In [56]:
item = uds[0]
print(item.keys())
print(item['mix_magnitude'].shape)

dict_keys(['index', 'mix_magnitude', 'ideal_binary_mask', 'source_magnitudes', 'weights'])
torch.Size([626, 129, 1])


In [54]:
def visualize_and_embed(sources, y_axis='mel'):
    plt.figure(figsize=(10, 4))
    plt.subplot(111)
    nussl.utils.visualize_sources_as_masks(
        sources, db_cutoff=-60, y_axis=y_axis)
    plt.tight_layout()
    plt.show()

    nussl.play_utils.multitrack(sources, ext='.wav')


visualize_and_embed(item['sources'])
print(item['metadata'])

KeyError: 'sources'

# Confusion Matrix

# Optimize

https://source-separation.github.io/tutorial/training/putting_it_all_together.html

In [7]:
BATCH_SIZE = 128



# Split the dataset into train and test sets
train_data, test_data = train_test_split(uds, test_size=0.25, random_state=0, shuffle=True)

print(f"There are {len(train_data)} samples in the train data")
print(f"There are {len(test_data)} samples in the test data")

There are 40 samples in the data
There are 30 samples in the train data
There are 10 samples in the test data


still left

In [ ]:
import torch
item = train_data[0]
for key in item:
    if torch.is_tensor(item[key]):
        item[key] = item[key].float().unsqueeze(0)
output = model(item)

In [ ]:
for key in output:
    print(key, type(output[key]), output[key].shape)

In [ ]:
model.verbose = True
output = model(item)

Training

In [8]:
import torch

nf = stft_params.window_length // 2 + 1
nac = 1
model = Mask_eliment.build(num_audio_channels = nf,
                           num_features = nac,
                           hidden_size = 50,
                           num_layers = 2,
                           bidirectional = True,
                           dropout = 0.1,
                           num_sources = 3,
                           activation = 'sigmoid')
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nussl.ml.train.loss.L1Loss()

def train_step(batch):
    optimizer.zero_grad()
    output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_magnitudes']
    )
    
    loss.backward() # backwards + gradient step
    optimizer.step()
    
    return loss.item() # return the loss for bookkeeping.

SeparationModel(
  (layers): ModuleDict(
    (model): Mask_eliment(
      (amplitude_to_db): AmplitudeToDB()
      (input_normalization): BatchNorm(
        (batch_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (recurrent_stack): RecurrentStack(
        (rnn): LSTM(257, 50, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
      )
      (embedding): Embedding(
        (linear): Linear(in_features=100, out_features=771, bias=True)
      )
    )
    (mask): Alias()
    (estimates): Alias()
  )
)
Number of parameters: 262273


In [13]:
# Comment out the line above to see the output
# of this cell in Colab or Jupyter Notebook
import tqdm

item = train_data[0] # Because of the transforms, this produces tensors.
batch = {} # A batch of size 1, in this case. Usually we'd have more.
for key in item:
    if torch.is_tensor(item[key]):
        batch[key] = item[key].float().unsqueeze(0)
    
N_ITERATIONS = 100
loss_history = [] # For bookkeeping

pbar = tqdm.tqdm(range(N_ITERATIONS))
for _ in pbar:
    loss_val = train_step(batch)
    loss_history.append(loss_val)
    pbar.set_description(f'Loss: {loss_val:.6f}')

  0%|          | 0/100 [00:16<?, ?it/s]


KeyError: 'mix_magnitude'

In [ ]:
plt.plot(loss_history)
plt.xlabel('# of iterations')
plt.ylabel('Training loss')
plt.title('Train loss history of our model')
plt.show()

In [16]:
def train_step(engine, batch):
    optimizer.zero_grad()
    output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_magnitudes']
    )
    
    loss.backward() # backwards + gradient step
    optimizer.step()
    
    loss_vals = {
        'L1Loss': loss.item()
    }
    
    return loss_vals # return the loss for bookkeeping.

In [17]:
def val_step(engine, batch):
    with torch.no_grad():
        output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_magnitudes']
    )    
    loss_vals = {'L1Loss': loss.item()}
    return loss_vals # return the loss for bookkeeping.

In [15]:
import nussl
import torch
from nussl.datasets import transforms as nussl_tfm
from pathlib import Path

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MAX_MIXTURES = int(1e8) # We'll set this to some impossibly high number for on the fly mixing.

stft_params = nussl.STFTParams(window_length=512, hop_length=128)

tfm = nussl_tfm.Compose([
    nussl_tfm.SumSources([['frog', 'hen', 'pig']]),
    nussl_tfm.MagnitudeSpectrumApproximation(),
    nussl_tfm.IndexSources('source_magnitudes', 1),
    nussl_tfm.ToSeparationModel(),
])

from torch.utils.data import DataLoader
from nussl_fram import dataset_v3
from sklearn.model_selection import train_test_split

BATCH_SIZE = 128

path = os.path.join(os.getcwd(), "audio")
uds = dataset_v3(path)
print(f"There are {len(uds)} samples in the data")

# Split the dataset into train and test sets
train_data, test_data = train_test_split(uds, test_size=0.25, random_state=0, shuffle=True)

print(f"There are {len(train_data)} samples in the train data")
print(f"There are {len(test_data)} samples in the test data")

# Create data loader for the train data
def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader

# Create train loader
train_dataloader = create_data_loader(train_data, BATCH_SIZE)

# Create test loader
test_dataloder = create_data_loader(test_data, BATCH_SIZE)

nf = stft_params.window_length // 2 + 1
model = Mask_eliment.build(nf, 1, 50, 1, True, 0.0, 1, 'sigmoid')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nussl.ml.train.loss.L1Loss()

def train_step(engine, batch):
    optimizer.zero_grad()
    output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_magnitudes']
    )
    
    loss.backward() # backwards + gradient step
    optimizer.step()
    
    loss_vals = {
        'L1Loss': loss.item(),
        'loss': loss.item()
    }
    
    return loss_vals

def val_step(engine, batch):
    with torch.no_grad():
        output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_magnitudes']
    )    
    loss_vals = {
        'L1Loss': loss.item(), 
        'loss': loss.item()
    }
    return loss_vals

# Create the engines
trainer, validator = nussl.ml.train.create_train_and_validation_engines(
    train_step, val_step, device=DEVICE
)

# We'll save the output relative to this notebook.
output_folder = Path('.').absolute()

# Adding handlers from nussl that print out details about model training
# run the validation step, and save the models.
nussl.ml.train.add_stdout_handler(trainer, validator)
nussl.ml.train.add_validate_and_checkpoint(output_folder, model, 
    optimizer, train_data, trainer, test_dataloder, validator)

trainer.run(
    train_dataloader, 
    epoch_length=10, 
    max_epochs=1
)

There are 40 samples in the data


Current run is terminating due to exception: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'nussl.core.audio_signal.AudioSignal'>
Engine run is terminating due to exception: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'nussl.core.audio_signal.AudioSignal'>


There are 30 samples in the train data
There are 10 samples in the test data


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'nussl.core.audio_signal.AudioSignal'>